In [17]:
import cv2
import numpy as np
from IPython.display import clear_output

import socket
import sys
import os
import time

video = cv2.VideoCapture(0)

track_sock = './track.sock'
arm_sock = './arm.sock'
    
sock = socket.socket(socket.AF_UNIX, socket.SOCK_STREAM)
sock.connect(track_sock)
sock2 = socket.socket(socket.AF_UNIX, socket.SOCK_STREAM)
sock2.connect(arm_sock)

t0 = 0

try:
    while True:
        ret, img = video.read()
        if not ret:
            break

        lower_hsv = np.array([100,43,46])
        upper_hsv = np.array([124,255,255])

        # 颜色遮罩
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv, lower_hsv, upper_hsv)

        # 锐化
        sharpen_kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
        sharpen = cv2.filter2D(mask, -1, sharpen_kernel)

        # 形态学变换
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
        close = cv2.morphologyEx(sharpen, cv2.MORPH_OPEN, kernel, iterations=2)

        # 提取边缘
        cnts = cv2.findContours(close, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = cnts[0] if len(cnts) == 2 else cnts[1]

        min_area = 2000
        max_area = 20000
        cnt = cnts[0]
        for c in cnts:
            area = cv2.contourArea(c)
            # 按大小筛选
            if area > min_area and area < max_area:
                cnt = c
                break
        x,y,w,h = cv2.boundingRect(cnt)
        cv2.putText(img, "center: ({}, {})".format(x+w/2, y+h/2), (x, y-20), cv2.FONT_HERSHEY_COMPLEX, 1, (218, 218, 3), 1)
        cv2.rectangle(img, (x, y), (x + w, y + h), (218,218,3), 2)
        clear_output(wait=True)
        print(x,y)
        if t0 == 0 and x >= 500 and y > 300 and y < 500:
            sock.sendall('stop'.encode())
            t0 = time.time()
        if t0 > 0 and time.time() - t0 > 1:
            sock2.sendall('{},{}'.format(x,y).encode())
            print(x,y)  
            break
except:
    pass
finally:
    video.release()
    sock.close()
    sock2.close()

551 394
551 394
